In [4]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd

from scipy import stats
from scipy.optimize import curve_fit
from scipy import asarray as ar,exp

import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np

import matplotlib.pyplot as plt

from networks import get_model

ModuleNotFoundError: No module named 'utils'

In [5]:
model = get_model(architecture='B_12-Wi_1024', resolution=64, num_classes=10,  checkpoint='in21k_cifar10')

NameError: name 'get_model' is not defined

### Build dataloader

In [ ]:
class RandomNoiseDataset(Dataset):
    def __init__(self, num_images, image_size):
        self.num_images = num_images
        self.image_size = image_size

    def __len__(self):
        return self.num_images

    def __getitem__(self, idx):
        # Create a random noise image
        image = np.random.rand(self.image_size[0], self.image_size[1], 3)
        # Convert the numpy array to a PyTorch tensor
        image = torch.from_numpy(image).float()
        # Permute the tensor to have the channel as the first dimension
        image = image.permute(2, 0, 1)
        # Set requires_grad to True
        image.requires_grad_(True)
        return image


In [ ]:
# Create the dataset
dataset = RandomNoiseDataset(num_images=1000, image_size= (64, 64))

# Create the data loader
data_loader = DataLoader(dataset, batch_size=1, shuffle=True)

# Now you can iterate over the data_loader to get batches of random noise images
data_loader

In [ ]:
# Function to visualize an image from the data loader
def visualize_random_noise_image(data_loader):
    # Get a batch of images
    images = next(iter(data_loader))
    # Take the first image from the batch
    image = images[0]
 # Detach the image from the computation graph and convert to numpy for visualization
    image = image.detach().cpu().numpy()
    # Permute the numpy array to have the channel as the last dimension
    image = np.transpose(image, (1, 2, 0))
    # Plot the image
    plt.imshow(image)
    plt.axis('off') # Turn off axis
    plt.show()

In [ ]:
visualize_random_noise_image(data_loader)

### Computing Gradients 

In [ ]:
gradients_across_batches = []
for images in data_loader:

    # Create a new tensor with the same data but reshaped, ensuring it's a leaf
    input = torch.tensor(images.clone().view(images.size(0), -1), requires_grad=True)
    input_gradients = torch.zeros_like(input[0])

    for i in range(1024):
        #predictions = model.linear_in(input) 
        predictions = model(input)
        loss = torch.sum(predictions[:,i])

        loss.backward()

        input_gradients += torch.mean(input.grad, axis=0)**2

        input.grad.zero_()
    
    gradients_across_batches = [input_gradients] 


    break



In [ ]:
stacked_gradients = torch.stack(gradients_across_batches, dim=0) 
mean_gradient = torch.mean(stacked_gradients, dim=0)

In [ ]:
mean_gradient=np.array(mean_gradient).reshape(3, 64, 64)

In [ ]:
channel1=mean_gradient[0, :, :]
channel2=mean_gradient[1, :, :]
channel3=mean_gradient[2, :, :]

### Visualize results

In [ ]:
plt.imshow(np.sqrt(channel3), cmap='magma', vmin=0)
plt.title('Norm of weights per pixel for channel 2 - all layers', size=11)
plt.colorbar()